In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

# domain specific libraries from which to import data
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# download the training data
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# download the test data
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
# iterating and visualizing the data
labels_map = {
    0:'T-Shirt',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8:'Bag',
    9:'Ankle Boot'
}

figure = plt.figure(figsize=(8,8))
cols, rows = 3,3
for i in range(1,cols*rows+1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows,cols,i)
    plt.title(labels_map[label])
    plt.axis('off')
    plt.imshow(img.squeeze(), cmap='gray')
plt.show()

In [4]:
# pass the dataset as an argument to 'Dataloader'. this wraps an iterable around our dataset, supporting automatic batching, sampling, shuffling
# and multiprocess data loading

# each element in the dataloader iterable will return a batch of 64 features and labels
batch_size = 64

# create data loaders 
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# get cpu or gpu device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

# define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
# to train a model, we need a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
# in a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the
# prediction error to adjust the models parameters.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch *len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [10]:
# check the models performance against the test dataset to ensure it is learning

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n')

In [11]:
# the training process is conducted over several epochs. during each epoch the model learns parameters to make better predictions.

# we want to see the models accuracy increase and the loss decrease at each epoch.

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n ----------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done!') 

Epoch 1
 ----------------------
loss: 2.296827 [    0/60000]
loss: 2.292490 [ 6400/60000]
loss: 2.273292 [12800/60000]
loss: 2.283179 [19200/60000]
loss: 2.257912 [25600/60000]
loss: 2.230037 [32000/60000]
loss: 2.244247 [38400/60000]
loss: 2.208177 [44800/60000]
loss: 2.205584 [51200/60000]
loss: 2.191633 [57600/60000]
Test error: 
 Accuracy: 36.6%, Avg Loss: 2.180246 

Epoch 2
 ----------------------
loss: 2.176276 [    0/60000]
loss: 2.177645 [ 6400/60000]
loss: 2.127500 [12800/60000]
loss: 2.157085 [19200/60000]
loss: 2.108216 [25600/60000]
loss: 2.050895 [32000/60000]
loss: 2.086300 [38400/60000]
loss: 2.012347 [44800/60000]
loss: 2.010074 [51200/60000]
loss: 1.961444 [57600/60000]
Test error: 
 Accuracy: 55.4%, Avg Loss: 1.958716 

Epoch 3
 ----------------------
loss: 1.967094 [    0/60000]
loss: 1.958430 [ 6400/60000]
loss: 1.856267 [12800/60000]
loss: 1.904567 [19200/60000]
loss: 1.793150 [25600/60000]
loss: 1.738528 [32000/60000]
loss: 1.763678 [38400/60000]
loss: 1.662681 [4

In [12]:
# saving the model
torch.save(model.state_dict(), 'model.pth')
print('The saved PyTorch model state to model.pth')

The saved PyTorch model state to model.pth


In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [20]:
# the model can be used to make predictions
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
